In [1]:
#
# Technical cell, run it first.
#

%matplotlib inline

import pandas as pd
from Bio import pairwise2
import seaborn
import matplotlib.pyplot as plt
import os

In [20]:
#
# This cell is just for testing purposes, you can safely skip it.
#


# TODO: merge D1 and D2
# Dsegments = {"D1": ("GAAATAGT", 4), 
#              "D2": ("CCTTCCTAC", 4), 
#              "D3": ("ACTGGGGGATACG", 4), 
#              "D2D3": ("CCTTCCTACACTGGGGGATACG", 6)}
filepath = "./data/britain/vdj.10_TRD_full.txt"
df = pd.read_csv(filepath, sep = "\t")
df.head()

,#count,freq,cdr3nt,cdr3aa,v,d,j,VEnd,DStart,DEnd,JStart
0,123727,0.486201,TGTGCCTGCCGATTTGGGTACTGGGGGTCCACCGATAAACTCATCTTT,CACRFGYWGSTDKLIF,TRDV2,TRDD3,TRDJ1,7,19,26,29
1,45214,0.177674,TGTGCTCTTGGGGAACGTTCCCTTCCTGCCAAACGATATGGGGTGC...,CALGERSLPAKRYGVLGISLPYTDKLIF,TRDV1,TRDD3,TRDJ1,15,47,53,64
2,23672,0.093022,TGTGCCTCCCTAGGCCTTCCTACGGGGCTGGGGGAAAAACTCATCTTT,CASLGLPTGLGEKLIF,TRDV2,TRDD2,TRDJ1,6,14,22,33
3,19940,0.078357,TGTGCTCTTGGGGAGAGCCCCTGGAGGTCAAACTGTGTATTAAAAA...,CALGESPWRSNCVLKTTHLTAQLFF,TRDV1,.,TRDJ2,13,-1,-1,53
4,18225,0.071617,TGTGCTCTTGGGGAACGGAGAGGGTACTGGGGTCCCACCGATAAAC...,CALGERRGYWGPTDKLIF,TRDV1,TRDD3,TRDJ1,15,25,31,35


In [8]:
#
# Main cell for alignment of D segments to ONLY ONE data frame WITHOUT restrictions. Specify your parameters below.
# To process a bunch of data frames, run this cell once and proceed to the next one.
#

# List of D gene segments in format "Dname": ("D seq", minimal length to align)
Dsegments = {"D1": ("GAAATAGT", 3), 
             "D2": ("CCTTCCTAC", 3), 
             "D3": ("ACTGGGGGATACG", 3), 
             "D2D3": ("CCTTCCTACACTGGGGGATACG", 5)}

# Filepath to the data frame for processing.
filepath = "./data/britain/vdj.10_TRD_full.txt"
filepath = "./data/holland//donor11.raw.cdrblast.txt"
filepath = "/Users/vdn/Projects/gd-project/data/dd35//donor11.raw.cdrblast.txt"

#
# Just run your cell now.
#
#---------------------------


df = pd.read_csv(filepath, sep = "\t")


def cut_ends(a):
    res = ""
    for i in range(len(a[0])):
        if a[0][i] == a[1][i]:
            res += a[1][i]
        elif a[1][i] == "-":
            res += "-"
        else:
            res += "*"
    return (a[0], res, a[2], a[3])


def pretty(a):
    print(a[0])
    print(a[1])
    

def get_all_alignments(seq, Dsegments, min_len):
    
    def get_start_end_len(a):
        start = -1
        end = -1
        
        max_start, max_end, max_len = -1, -1, -1
        start_pos = 0
        while (a[0][start_pos] == "-"):
            start_pos += 1
        for i in range(start_pos, len(a[0])):
            if a[0][i] == "-" or a[1][i] == "-":
                if end - start + 1 > max_len:
                    max_start = start
                    max_end = end
                    max_len = end - start + 1
                start = -1
                end = -1
            else:
                if a[0][i] == a[1][i]:
                    if start == -1:
                        start = i
                    end = i
                else:
                    if end - start + 1 > max_len:
                        max_start = start
                        max_end = end
                        max_len = end - start + 1

                    start = -1
                    end = -1
        
        return max_start - start_pos, max_end - start_pos, max_len
    
    D_alignments = []
    for D_name, (D, length) in Dsegments.items():
        a = pairwise2.align.globalxs(seq, D, -3, -3, penalize_end_gaps = False)[0]
        cur_start, cur_end, cur_len = get_start_end_len(a)
#         print(D_name, D)
#         pretty(cut_ends(a))
#         print(cur_start, cur_end, cur_len)
        if cur_len >= length:
            D_alignments.append((cur_start, cur_end, cur_len, D_name))
            
    D_alignments = sorted(D_alignments, reverse = True, key = lambda x: x[3])
    D_alignments = sorted(D_alignments, reverse = True, key = lambda x: len(x[3]))
    D_alignments = sorted(D_alignments, reverse = True, key = lambda x: x[2])
    return D_alignments


def find_the_longest(seq, start, end, Dsegments, min_len):
    if end - start > min_len - 1:
        res = get_all_alignments(seq[start:end], Dsegments, min_len)
#         print("Alignment:\t", res)
        # if the best score is D2D3,
        # check if D2D3 score better than D2 or D3
        if res:
            if res[0][3] == "D2D3":
                if len(res) == 1:
                    return [res[0]]
                if res[0][2] > res[1][2]:
                    # if so then this is true D2D3 without N alignment
                    return [res[0]]
                else:
                    # otherwise it's just one of the D2, D3 or D2-N-D3 alignment
                    # so just pop it
                    res.pop(0)

            if res:
                # if the best score isn't D2D3,
                # then check for dual alignments
                return sorted((filter(lambda x: x[2] == res[0][2], res)), key = lambda x: x[0])

    return []


def align_df(df, Dsegments, min_len):
    df["D.new"] = "NA"
    df["D5.new1"] = -1
    df["D3.new1"] = -1
    df["D5.new2"] = -1
    df["D3.new2"] = -1
    df["D5.new3"] = -1
    df["D3.new3"] = -1
    for i in range(len(df)):
        alg = find_the_longest(df["cdr3nt"][i], df["VEnd"][i] + 1, df["JStart"][i], Dsegments, min_len)
        if alg:
            df.loc[i, "D.new"] = ""
            for j, a in enumerate(alg):
                df.loc[i, "D5.new" + str(j+1)] = df["VEnd"][i] + 1 + a[0]
                df.loc[i, "D3.new" + str(j+1)] = df["VEnd"][i] + 1 + a[1]
                df.loc[i, "D.new"] += ("," if j != 0 else "") + "TRD" + a[3]

                
align_df(df, Dsegments, 3)

In [34]:
#
# Main cell for alignment of D segments to MANY data frames WITHOUT restrictions. Specify your parameters below.
#

# List of D gene segments in format "Dname": ("D seq", minimal length to align)
Dsegments = {"D1": ("GAAATAGT", 3), 
             "D2": ("CCTTCCTAC", 3), 
             "D3": ("ACTGGGGGATACG", 3), 
             "D2D3": ("CCTTCCTACACTGGGGGATACG", 5)}

# Specify your first folder with the data
folder_path = "./data/britain/"
files1 = [folder_path + x for x in os.listdir(folder_path) if not x.startswith(".")]

# Specify your second folder with data. 
Put folder_path = "" if you don't have only one folder.
folder_path = "./data/holland/"
if folder_path: files2 = [folder_path+ x for x in os.listdir(folder_path) if not x.startswith(".")]

files = files1 + files2

# Specify the output postfix for processed files.
# E.g., input file "data.txt" processed with postfix "dd35" 
# will result in "data.dd35.txt".
postfix = "dd35"

#
# Just run your cell now.
#
#---------------------------


for i, file in enumerate(files):
    df = pd.read_csv(file, sep = "\t")
    align_df(df, Dsegments, 4)
    out_file = file[:file.find("/", 2)+1] + postfix + "/" + file[file.rfind("/") + 1:]
    df.to_csv(out_file, sep = "\t", index = False)

In [35]:
Dsegments = {"D1": ("GAAATAGT", 4), 
             "D2": ("CCTTCCTAC", 4), 
             "D3": ("ACTGGGGGATACG", 4), 
             "D2D3": ("CCTTCCTACACTGGGGGATACG", 6)}

folder_path = "./data/britain/"
files1 = [folder_path + x for x in os.listdir(folder_path) if not x.startswith(".")]
folder_path = "./data/holland/"
files2 = [folder_path+ x for x in os.listdir(folder_path) if not x.startswith(".")]
files = files1 + files2
# files = files[:2]
stat_df = pd.DataFrame()
stat_df["Subject"] = ""
stat_df["D"] = ""
# f, axes = plt.subplots(len(files) // 2 + 1, 2, figsize=(18, 10))
for i, file in enumerate(files):
    df = pd.read_csv(file, sep = "\t")
    align_df(df, Dsegments, 4)
    out_file = file[:file.find("/", 2)+1] + "dd46/" + file[file.rfind("/") + 1:]
    df.to_csv(out_file, sep = "\t", index = False)
#     seaborn.factorplot(x = "D.new", data = df, kind = "count", size = 3, aspect = 5)#, ax = axes[i // 2, i % 2])
#     g.fig.get_axes()[0].set_yscale('log')
#     axes[i // 2, i % 2].set_title(file)
# plt.show()

In [1]:
seq1 = "TGTGCTTTT"
seq2 = "GTGTGGGGAGA"
a = pairwise2.align.localms(seq1, seq2, 2, -1, -3, -3, penalize_end_gaps = False)[0]
pretty(cut_ends(a))
print(a[2:])
    
if ():
    print("!")
    
df["newcol"] = 1
df

stat_df


NameError: name 'pairwise2' is not defined

In [ ]:
list(filter(lambda x: x.find(",") != -1, list(df['D.new'])))